In [33]:
import pandas as pd
import numpy as np

### Load Files & Create DF

In [34]:
brands_file = 'data/brand_lines.xlsx'
market_system_file = 'data/market_system.xlsx'
market_trip_file = 'data/market_trip.xlsx'
raw_data_file = 'data/raw_data.xlsx'
brands_df = pd.read_excel(brands_file)
market_system_df = pd.read_excel(market_system_file)
market_trip_df = pd.read_excel(market_trip_file)
raw_data_df = pd.read_excel(raw_data_file)

# Brands

### Remove Unnamed Columns

In [35]:
for col in brands_df.columns:
    if 'Unnamed' in col:
        del brands_df[col]

### Melt Columns

In [36]:
brands_df = pd.melt(brands_df)
brands_group = brands_df.groupby(['variable', 'value']).count().reset_index()

In [37]:
brands_group = brands_group[brands_group['value'] != 'Total Lines']

In [38]:
brands_group.head()

,variable,value
0,58 Degrees & Holding,Big Stump Brew Co Lions Crown
1,58 Degrees & Holding,Big Stump Brewing Rowan's Red Ale
2,58 Degrees & Holding,Matua Sauvignon Blanc
3,58 Degrees & Holding,Powder Keg Cabernet Sauvignon
5,58 Degrees & Holding,Two Rivers Ginger Cider


# Market System Builder

Market System Builder Represents what's currently in the market. It needs to be audited before billing the distributors.
Historical reports (brands by distributor, raw data reports, and market trip reports) are compared against the a market system builder to find red flags.

In [39]:
market_system_df.head()

,distributor,serviceable,distactive,locid,city,address1,zip,tapname,bid,zonevalue,area,did,client,clientNumber,recurDescription,quantity
0,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Amador Brewing Ipa,1037280,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1
1,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Amador Brewing Kolsch,1057032,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1
2,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Big Stump Brewing Rowan's Red Ale,1209098,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1
3,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Fort Rock Foggy Goggles,1092374,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1
4,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Matua Sauvignon Blanc,1153068,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1


# FLAGS 1 & 2

### Check if Tap is Associated with Distributor

* Throw flag if the tap is not associated with distributor in Brand Lines by Dist Report
* If tap is not associated with distributor, check if the brewery is associated with distributor


Checking if the tap's brewery is associated with the distributor eliminates hundreds of taps from a manual audit. We assume that if the distributor is associated with other taps from that brewery, it should also be associated with the new tap.

In [40]:

## Iterrate through every row of market_system
for index, row in market_system_df.iterrows():
    
    tap = row['tapname']
    dist = row['distributor']
    
    ## Create Data Frames for market system, brands, and raw data that only include the row's distributor
    market_system_audit = market_system_df[market_system_df['distributor'] == dist]
    brand_audit = brands_group[brands_group['variable'] == dist]
    brewery_audit = raw_data_df[raw_data_df['Distributor'] == dist]
    
    ## List of unique taps that have historically been associated with distributor
    brand_taps = brand_audit['value'].unique()
    
    ## List of unique breweries that have historically been associated with the distributor
    breweries = brewery_audit['BreweryName'].unique()
    
    ## Empty list that will store a certain portion of a breweries name.
    ## Used to check if the tap has a breweries name in it.
    brewery_names = []
    
    ## 1. Loop through breweries associated with distributor
    ## 2. Grab the first word in the brewery's name and add it to the brewery_names list
    for brewery in breweries:
        if not isinstance(brewery, float):
            name = brewery.split()
        
            if len(name[0]) > 3:
                brewery_names.append(name[0])
                
    ## 1. Check if the current row tap is associated with the distributor
    ## 2. If it's not, flag the row
    
    if tap not in brand_taps:
        market_system_df.loc[index, 'dist_tap_audit'] = 'x'
        
    ## 3. If the tap is flagged, check to see if the brewery name appears in the tap's name.
    ## 4. If it does, flag the row
       
        if any(x in tap for x in brewery_names):
            market_system_df.loc[index, 'brewery_dist_audit'] = 'x'
            
    ## THE SECOND FLAG IS A GOOD FLAG
    ## It means that the brewery is associated with the distributor, and we probably bill for it.
    ## Any flags in the 'dist_tap_audit' column that do not have 'brewery_dist_audit' flagged as well, 
    ### will need to be manually audited
    
        

#### brewery_name

The brewery_names list only stores the first part of the breweries name as long as the length of that string is greater than 3.


In [41]:
brewery_names

['Camelot', 'Line', 'Angeline', 'Craft', 'Girls']

## Tap Audit Results

In [42]:
market_system_df[['dist_tap_audit','brewery_dist_audit']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19145 entries, 0 to 19144
Data columns (total 2 columns):
dist_tap_audit        562 non-null object
brewery_dist_audit    353 non-null object
dtypes: object(2)
memory usage: 299.2+ KB


In [43]:
tap_audit = market_system_df[['distributor', 'client', 'tapname', 'dist_tap_audit', 'brewery_dist_audit']]
tap_audit = tap_audit[(tap_audit['dist_tap_audit'] == 'x') | (tap_audit['brewery_dist_audit'] == 'x')]
tap_audit.head()

,distributor,client,tapname,dist_tap_audit,brewery_dist_audit
0,58 Degrees & Holding,58 Degrees & Holding Co. (Sacramento/18th),Amador Brewing Ipa,x,NaN
1,58 Degrees & Holding,58 Degrees & Holding Co. (Sacramento/18th),Amador Brewing Kolsch,x,NaN
3,58 Degrees & Holding,58 Degrees & Holding Co. (Sacramento/18th),Fort Rock Foggy Goggles,x,NaN
19,ALTAMONT BEER WORKS,2M - Aptos St BBQ (Aptos/Aptos),Altamont Beer Works Vasco Juice,x,x
124,ALTAMONT BEER WORKS,Catos Ale House (Oakland/Piedmont),Altamont Beer Works L Town,x,x


#### Total Taps to Audit:

In [44]:
total_tap_audit = tap_audit[tap_audit['brewery_dist_audit'] != 'x'].count()['tapname']

print(f'{total_tap_audit} total taps to audit.')

209 total taps to audit.


In [45]:
market_system_df.head()

,distributor,serviceable,distactive,locid,city,address1,zip,tapname,bid,zonevalue,area,did,client,clientNumber,recurDescription,quantity,dist_tap_audit,brewery_dist_audit
0,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Amador Brewing Ipa,1037280,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1,x,NaN
1,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Amador Brewing Kolsch,1057032,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1,x,NaN
2,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Big Stump Brewing Rowan's Red Ale,1209098,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1,NaN,NaN
3,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Fort Rock Foggy Goggles,1092374,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1,x,NaN
4,58 Degrees & Holding,True,True,1027267,Sacramento,1217 18th Street,95811,Matua Sauvignon Blanc,1153068,CA05,NaN,1002166,58 Degrees & Holding Co. (Sacramento/18th),NaN,Clean and bill all IND lines to acct\nback doo...,1,NaN,NaN


# FLAGS 3 & 4

### Check if CA01 / CA04 accounts are zoned correctly

* Import geojson polygon of zones CA01 and CA04
* Check if zone CA01's account coordinates lie within the polygon of CA04
* Check if zone CA04's account coordinates lie within the polygon of CA01
* If the account's coordinates lie within incorrect polygon, flag the account.


Zone CA01 and CA04 are both in San Jose and are divided by I-880. New accounts are often zoned incorrectly and billed to incorrect distributors. By checking against a polygonal representation of the zone, we can progmatically flag accounts that are zoned incorrectly.

In [46]:
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas.geoseries import *
from shapely.geometry import Point, Polygon

%matplotlib inline

ImportError: cannot import name 'CRS' from 'pyproj' (C:\Users\Terra\Anaconda3\lib\site-packages\pyproj\__init__.py)

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)

In [ ]:
ca04_poly = gpd.read_file('data/ca04.geojson')
ca01_poly = gpd.read_file('data/ca01.geojson')



In [ ]:
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution='c',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
ax1 = ca04_poly.plot(color='purple', edgecolor = 'k', linewidth = 2)
ax2 = ca01_poly.plot(color='pink', edgecolor = 'k', linewidth = 2, ax=ax1)

In [ ]:
locations = pd.read_excel('data/BAY_Send_Orders.xlsx')

In [ ]:
ca04 = locations[locations[' Service Location - Contact - First Name'] == 'CA04']  
ca01 = locations[locations[' Service Location - Contact - First Name'] == 'CA01']

In [ ]:
ca04 = ca04[[' Service Location - Description',' Latitude', ' Longitude']]
ca01 = ca01[[' Service Location - Description',' Latitude', ' Longitude']]

In [ ]:

ax1 = ca04_poly.plot(color='None', edgecolor = 'k', linewidth = 2, label="Zone CA04 Shape")
ax2 = ca01_poly.plot(color='None', edgecolor = 'k', linewidth = 2, ax=ax1)
ax3 = ca04.plot(kind='scatter', x=' Longitude', y = ' Latitude', color = 'g', ax=ax1, label="Zone CA04 Account", alpha = 0.8)
ax4 = ca01.plot(kind='scatter', x=' Longitude', y = ' Latitude', color = 'b', ax=ax1, label ="Zone CA01 Account", alpha = 0.8)
plt.savefig('CA04-CA01 Zone Errors')

In [ ]:
polygon4 = pd.read_json('data/ca04.geojson')
polygon4 = np.array(polygon4['features'][0]['geometry']['coordinates'][0])

polygon1 = pd.read_json('data/ca01.geojson')
polygon1 = np.array(polygon1['features'][0]['geometry']['coordinates'][0])

In [ ]:
poly4 = Polygon(polygon4)
poly1 = Polygon(polygon1)

In [ ]:
for index,row in ca01.iterrows():
    point = Point(row[' Longitude'], row[' Latitude'])
    if point.within(poly4):
        ca01.loc[index, 'ca01_audit'] = 'x'

for index,row in ca04.iterrows():
    point = Point(row[' Longitude'], row[' Latitude'])
    if point.within(poly1):
        ca04.loc[index, 'ca04_audit'] = 'x'


In [ ]:
ca01errors = ca01[ca01['ca01_audit'] == 'x']
ca01errors = ca01errors[[' Service Location - Description', 'ca01_audit']]
ca01errors.rename(columns = {' Service Location - Description':'client'}, inplace = True) 
ca01errors.drop_duplicates(inplace=True)

In [ ]:
ca04errors = ca04[ca04['ca04_audit'] == 'x']
ca04errors = ca04errors[[' Service Location - Description', 'ca04_audit']]
ca04errors.rename(columns = {' Service Location - Description':'client'}, inplace = True) 
ca04errors.drop_duplicates(inplace=True)

In [ ]:
market_system_df = pd.merge(market_system_df, ca01errors, how='left', on='client')
market_system_df = pd.merge(market_system_df, ca04errors, how='left', on='client')

In [ ]:
market_system_df[(market_system_df['ca01_audit'] == 'x') | (market_system_df['ca04_audit'] == 'x')]

In [ ]:
ca04errors

In [ ]:
result